# Convert the .sqlite3 files for application


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

In [17]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd
from tzwhere import tzwhere
from dateutil import tz
from datetime import datetime as dt
tzg = tzwhere.tzwhere()
db = 'E:/Top_Users_Timelines/Austria_timelines.sqlite3'

def where_self(row):
    try:
        x = tz.gettz(tzg.tzNameAt(row["latitude"], row["longitude"]))
    except:
        x = "Unknown"
    return x

def where2time(row):
    from_zone = tz.gettz('UTC')
    timezone = row["time_zone"]

    rawUTC = dt.strptime(row["created_at"],'%b %d %H:%M:%S %Y')# , '%b %d %H:%M:%S %Y'
    utc = rawUTC.replace(tzinfo=from_zone)
    central = str(utc.astimezone(timezone))
    rawT = dt.strptime(central[:-6],'%Y-%m-%d %H:%M:%S')
    return rawT

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## 1 Load data from the raw .sqlite3

In [3]:
conn = sqlite3.connect(db)
df_raw = pd.read_sql_query("SELECT * FROM geo INNER JOIN records ON geo.rec_id = records.id", con=conn)
df_raw = df_raw.loc[:, ['tw_id', 'user_id', 'time',
                        'coord_lat', 'coord_long']].rename(columns={'tw_id': 'tweet_id',
                                                                    'time': 'created_at',
                                                                    'coord_lat': 'latitude',
                                                                    'coord_long': 'longitude'})

## 2 Get time zone and convert the UTC time into local time

In [9]:
df_raw["time_zone"] = df_raw.apply(lambda row: where_self(row), axis = 1)
df_raw.iloc[0]

tweet_id       1176080703810875392
user_id                          1
created_at    Sep 23 10:27:44 2019
latitude                   25.0361
longitude                  121.555
time_zone            tzfile('ROC')
Name: 0, dtype: object

In [14]:
df_raw = df_raw.loc[df_raw["time_zone"] != "Unknown",:]
df_raw["created_at"] = df_raw.apply(lambda row: where2time(row), axis = 1)
df_raw["time_zone"] = df_raw["time_zone"].apply(lambda x: str(x)[8:-2])

In [15]:
df_raw.loc[:, 'month'] = df_raw.loc[:, 'created_at'].apply(lambda x: x.month)
df_raw.loc[:, 'weekday'] = df_raw.loc[:, 'created_at'].apply(lambda x: x.weekday())
df_raw.loc[:, 'hour_of_day'] = df_raw.loc[:, 'created_at'].apply(lambda x: x.hour)
df_raw = df_raw.loc[df_raw['time_zone']!='']

## 3 Save into a new .sqlite3

In [25]:
engine = create_engine('sqlite:///' + ROOT_dir + '/dbs/austria/austria.sqlite3', echo=True)
sqlite_connection = engine.connect()
sqlite_table = 'geo_tweet'
df_raw.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)
sqlite_connection.close()

2020-09-23 18:21:38,775 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-23 18:21:38,776 INFO sqlalchemy.engine.base.Engine ()
2020-09-23 18:21:38,777 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-23 18:21:38,777 INFO sqlalchemy.engine.base.Engine ()
2020-09-23 18:21:39,014 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("geo_tweet")
2020-09-23 18:21:39,014 INFO sqlalchemy.engine.base.Engine ()
2020-09-23 18:21:39,032 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE geo_tweet (
	"index" BIGINT, 
	tweet_id TEXT, 
	user_id BIGINT, 
	created_at DATETIME, 
	latitude FLOAT, 
	longitude FLOAT, 
	time_zone TEXT, 
	month BIGINT, 
	week_day BIGINT, 
	hour_of_day BIGINT
)


2020-09-23 18:21:39,033 INFO sqlalchemy.engine.base.Engine ()
2020-09-23 18:21:39,161 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-23 18:21:39,164 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_geo_twee